# Introduction to Langchain Tools

In this notebook , we are going to look at what Tools are in Langchain and how to use a model to call tools.

A tool in Langchain is a construct , that allows a LLM or a chain to interact with something external. In simple terms , a tool is a function that has a purpose and performs some kind of an action or a task.
For example, the tool can search the internet for information or it can do some math or even it can be a function that acts as a retriever.

In a very basic form, a tool should have the following components:

* The name of the tool
* A description of what the tool does
* JSON formatted schema of what the inputs to the tools are
* A function to call when the tool is invoked

Once a tool is defined, we can use the power of LLM to call the tool with appropriate inputs. The name, description and JSON schema are provided as context to the LLM, allowing the LLM to determine how to use the tool appropriately.

Lets look at an example of defining a simple tool

### Defining a  basic Tool

In [12]:
from langchain_core.tools import tool

@tool
def upper_case(string: str) -> str:
    """ convert lower case string of characters to upper case"""
    return string.upper()

@tool
def int_to_hex(integer: int) -> str:
    """A function that converts a given integer to hexa decimal"""
    return hex(integer)

#Another way of defining tool by simply defining the function
def join_elements(a: str, b:str) -> str:
    """A function that joins two strings"""
    return ''.join(a, b)

tools = [upper_case, int_to_hex, join_elements]

### Defining tool using Pydantic(optional)

In [13]:
from langchain_core.pydantic_v1 import BaseModel, Field

class UpperCase(BaseModel):
   """ convert lower case string of characters to upper case"""

   lcase_string: str = Field(..., description="lower case string that needs to be converted")

class IntToHex(BaseModel):
    """converts an integer to its corresponding hexa decimal value"""

    int_to_hex: str = Field(..., description="integer value that needs to be converted to hex")

#tools = [UpperCase, IntToHex]


Note that the docstrings here are crucial, as they will be passed as context to the model along with the class name.

Once tools are defined , we can now provide the list of available tools to the LLM along with a prompt. Based on the user prompt, a LLM model will then invoke one or more of the available tools with appropriate inputs.

we use "bind_tools()" method to pass available tools to the model. Under the hood , "bind_tools()" is an interface that can be used with LLMs supporting tool calling feature from different vendors such as openai, anthropic, google , azure and more. For our demo , we are going to use openai llm model.

Before using a LLM for calling tools , following things are important to consider:
* Chat models that have been fine-tuned for tool calling will be better at tool calling than non-fine-tuned models.
* Models will perform better if the tools have well-chosen names, descriptions, and JSON schemas.
* The purpose of tool calling is simply to request LLM (based on the user query or prompt) on selecting the best tools to use from the list of available tools. It is not intended to making the function call with the arguments and returning the output from function call.

In [1]:
import os, getpass
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = getpass.getpass()

llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

llm_with_tools.invoke("convert integer 10 to hex")


NameError: name 'tools' is not defined

As we can see in the cell output above, the AI response doesnt give us the answer to user question. Instead it shows which tool it will use based on the user query along with properly formatted arguments to the selected tool

if the output has attribute "tool_calls" , we can print it to see which tool AI is selecting along with the arguments based on the user query

In [15]:
#llm_with_tools.invoke("convert integer 10 to hex").tool_calls
llm_with_tools.invoke("join the strings Apple and Banana")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cDvf5rX4S5JzhPvKkq2Qu0Sq', 'function': {'arguments': '{\n  "a": "Apple",\n  "b": "Banana"\n}', 'name': 'join_elements'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 124, 'total_tokens': 147}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f89e0195-98c5-4106-954b-3906592994a2-0', tool_calls=[{'name': 'join_elements', 'args': {'a': 'Apple', 'b': 'Banana'}, 'id': 'call_cDvf5rX4S5JzhPvKkq2Qu0Sq'}])